In [1]:
!pip install gradio openai pillow


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 1. open ai 키 호출

In [2]:
import gradio as gr
from openai import OpenAI
from PIL import Image
import io
import os
import base64

# OpenAI API 클라이언트 초기화
client = OpenAI(
    api_key = ''
)

### 2. 텍스트 요약
- 업로드된 텍스트 파일을 요약해주는 함수 선언

In [3]:
def summarize_text(file):
    with open(file.name, "r", encoding="utf-8") as f:
        text = f.read()

    prompt = f'''
    **Instructions** :
    - Summarize the given text into 3 bullet points in Korean.
    - Focus on key arguments and avoid redundant information.
    - Use concise but informative language.
    -text : {text}
    '''

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text in Korean."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.1,
        top_p=0.9,
    )

    return response.choices[0].message.content

### 3. 요약된 텍스트 번역
- 번역해주는 함수 선언

In [12]:
# 번역 함수
def translate(text, target_language):
    if not text:
        return ""

    prompt = f"You are a helpful translator. Please do not write any sentences other than the translation result. Translate the text sent by the user into {target_language}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ],
        max_tokens=1000,
        temperature=0.1,
        top_p=0.9,
    )

    return response.choices[0].message.content

### 4. 요약된 텍스트를 기반으로 하나의 이미지로 표현
- 이미지를 생성하는 모델(DALLE) 이용
- 이미지 생성 함수 선언

In [13]:
def generate_image(text):
    response = client.images.generate(
        model="dall-e-3",
        prompt=text,
        size="1024x1024"
    )
    image_url = response.data[0].url

    return image_url

### 5. 이미지를 묘사
- 이미지를 이해하는 모델(GPT4V)이용
- describe 함수 선언

In [14]:
# OpenAI GPT4V
# 이미지 이해 API
def describe_image(text):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해서 아주 자세히 묘사해줘"},
            {
            "type": "image_url",
            "image_url": {
                "url": text,
            },
            },
        ],
        }
    ],
    max_tokens=1024,
    )
    return response.choices[0].message.content

### 6. 묘사된 텍스트를 음성으로
- tts함수 선언

In [15]:
# OpenAI TTS API
# 텍스트 -> 음성
def TTS(response):
    response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=response
    )
    filename = "output.mp3"
    response.stream_to_file(filename)
    return filename

### 7. 기능별로 묶기
- 요약과 번역
- 이미지 생성과 묘사

In [16]:
def summarize_and_translate(file, target_language):
    summary = summarize_text(file)
    translated_text = translate(summary, target_language)
    return summary, translated_text

def generate_image_and_description(translated_text):
    image_url = generate_image(translated_text)
    image_description = describe_image(image_url)
    return image_url, image_description

### 8. 그라디오 인터페이스 구성

In [17]:
with gr.Blocks(css=".gradio_container {background-color: #f9f9f9;}") as demo:
    gr.Markdown(
        """
        # 🗒️ 텍스트 파일만 업로드 해봐요!
        **요약** 과 **번역**
        **이미지 생성** 및 **음성변환**까지 해드려요.
        """
    )

    # 파일 업로드 & 언어 선택
    with gr.Row():
        file_input = gr.File(label="📂 텍스트 파일 업로드")
        lang_input = gr.Dropdown(choices=["한국어", "영어", "일본어"], label="🌎 번역 언어 선택", value="한국어")

    # 요약 & 번역 출력
    with gr.Row():
        summarize_output = gr.Textbox(label="📜 3줄 요약", interactive=False)
        translate_output = gr.Textbox(label="🌍 번역된 텍스트", interactive=False)

    # 이미지 생성 & 설명 출력
    with gr.Row():
        image_output = gr.Image(label="🖼️ 생성된 이미지")
        description_output = gr.Textbox(label="📝 이미지 설명", interactive=False)

    # 음성 변환 출력
    with gr.Row():
        audio_output = gr.Audio(label="🔊 음성 변환 결과")

    # 실행 버튼
    summarize_translate_button = gr.Button("📜 요약 및 번역 실행")
    image_generate_button = gr.Button("🎨 이미지 생성 및 묘사 실행")
    tts_button = gr.Button("🔊 음성 변환 실행")

    # 각 버튼 클릭 시 동작 이벤트
    summarize_translate_button.click(summarize_and_translate, inputs=[file_input, lang_input], outputs=[summarize_output, translate_output])
    image_generate_button.click(generate_image_and_description, inputs=[translate_output], outputs=[image_output, description_output])
    tts_button.click(TTS, inputs=[description_output], outputs=[audio_output])

# 디버그 모드로 실행
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://5e02c01b8786762907.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\user\AppData\Local\Temp\ipykernel_18772\2130974533.py:10: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e02c01b8786762907.gradio.live
